In [1]:
from pyTsetlinMachineParallel.tm import MultiClassTsetlinMachine
from tmu.tsetlin_machine import TMCoalescedClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
import pickle
import torch
import tqdm
from collections import Counter, defaultdict
!export OMP_NUM_THREADS=10
n_clauses_per_class = 200
treshold = int(n_clauses_per_class * 0.8)
s_param = 6.8
s_param = 21.3
s_param = 11.40000000000001
"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [2]:
# tm = MultiClassTsetlinMachine(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)


In [3]:
rf = RandomForestClassifier()

In [3]:
game_amount = 100000
with open("/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), "rb") as fp:
        games = pickle.load(fp)
print(games[:3])
print(games[0][0])
games[0].shape

[[array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
         1., 0., 0., 0., 0., 0., 0.], dtype=float32)
  0]
 [array([1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
         0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1

(2,)

In [5]:
games[0][0].shape

(126,)

In [4]:
train_i = int(games.shape[0] * 0.9)
train_i

514835

/bin/bash: nvcc: command not found


In [143]:
ys = Counter()
for game in games:
    ys[game[1]] += 1
ys

Counter({0: 200687,
         4: 28227,
         10: 48887,
         9: 73117,
         12: 46446,
         11: 21266,
         13: 5810,
         1: 61140,
         3: 29043,
         6: 40378,
         5: 12874,
         7: 1322,
         2: 767,
         8: 918,
         15: 282,
         14: 396,
         16: 350,
         17: 91,
         18: 38})

In [5]:
x_train =np.array([game[0] for game in games[:train_i, :1]])
x_test =np.array([game[0] for game in games[train_i:, :1]])

y_train =np.array([game[0] for game in games[:train_i, 1:]])
y_test =np.array([game[0] for game in games[train_i:, 1:]])


# x_test = games[train_i:, :1]

# y_train = games[:train_i, 1:]
# y_test = games[train_i:, 1:]

# for game in games:
#     print(game[0])
#     break

In [10]:
y_train.shape

(514835,)

In [9]:
!CUDA_ROOT=/usr/local/cuda

In [10]:
epochs = 1
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

print("Accuracy tm:", 100*(cuda_tm.predict(x_test) == y_test).mean())

ExecError: error invoking 'nvcc --preprocess -arch sm_61 -I/home/jaoi/anaconda3/envs/rufus/lib/python3.8/site-packages/pycuda/cuda /tmp/tmp9mi4vlbl.cu --compiler-options -P': [Errno 2] No such file or directory: 'nvcc'

In [12]:
rf.fit(x_train, y_train)
print("Accuracy rf:", 100*(rf.predict(x_test) == y_test).mean())

Accuracy rf: 97.0124466820502


In [13]:
def tsetlin_hyperparameter_search(n_clauses_per_class, x_t, y_t, x_v, y_v):
    """
    Search for the best hyperparameters for the tsetlin machine
    """
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best = 0.0
    treshold = int(0.8 * n_clauses_per_class)
    print("starting training")
    for s_param in tqdm.tqdm(np.arange(1.0, 30.0, 0.1)):
        tm = MultiClassTsetlinMachine(n_clauses_per_class, treshold, s_param, weighted_clauses=True, boost_true_positive_feedback=0)
        tm.fit(x_t, y_t, epochs=200)
        acc = 100*(tm.predict(x_v) == y_v).mean()
        if acc > best:
            print("NEW BEST ACC {} WITH s_param {}".format(acc, s_param))
            best = acc
# tsetlin_hyperparameter_search(50, x_train, y_train, x_test, y_test)

In [14]:
tm.number_of_classes

19

In [15]:
def get_name_of_feature_from_3_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr == 17:
        return "self can bid bit"
    elif feature_nr in [18,19,20,21,22]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 18])
    elif feature_nr in [23,24,25,26,27]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 23])
    elif feature_nr in [28,29,30,31]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 28])
    elif feature_nr in [32,33]:
        return "next turn order {}".format(2**reverse_4_bits[feature_nr - 32])
    # for next player in line
    elif feature_nr in [34, 35, 36, 37, 38]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 34]))
    elif feature_nr in [39, 40, 41, 42]:
        return "next player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 39])
    elif feature_nr in [43, 44, 45, 46]:
        return "next player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47, 48, 49, 50]:
        return "next player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr == 51:
        return "next can bid bit"
    elif feature_nr in [52, 53, 54, 55, 56]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 52])
    elif feature_nr in [57, 58, 59, 60, 61]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 57])
    elif feature_nr in [62, 63, 64, 65]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 62])
    elif feature_nr in [66, 67]:
        return "next turn order {}".format(2**reverse_4_bits[feature_nr - 66])
    # for last player in line
    elif feature_nr in [68, 69, 70, 71, 72]:
        return "last player coins bit {}".format(2**(reverse_5_bits[feature_nr - 68]))
    elif feature_nr in [73, 74, 75, 76]:
        return "last player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 73])
    elif feature_nr in [77, 78, 79, 80]:
        return "last player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 77])
    elif feature_nr in [81, 82, 83, 84]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 81])
    elif feature_nr == 85:
        return "last player can bid"
    elif feature_nr in [86, 87, 88, 89, 90]:
        return "current next player bid {}".format(2**reverse_5_bits[feature_nr - 86])
    elif feature_nr in [91, 92, 93, 94, 95]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 91])
    elif feature_nr in [96, 97, 98, 99]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 96])
    elif feature_nr in [100, 101]:
        return "next turn order {}".format(2**reverse_4_bits[feature_nr - 100])
    # remainder stats
    elif feature_nr in [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]:
        return "{}. largest in stack".format(feature_nr - 101)
    elif feature_nr in [113, 114, 115, 116]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 113])
    elif feature_nr in [117, 118, 119, 120]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 117])
    elif feature_nr in [121, 122, 123, 124]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 121])
    elif feature_nr == 125:
        return "board_std"
    return "not implemented"
        

In [16]:
sum([5, 4, 4, 4, 1, 5, 5, 4, 2]) * 3 + 11 + 4 + 4 + 4 + 1

126

In [17]:
sum([5, 4, 4, 4, 1, 5, 5, 4, 2])

34

In [18]:
def show_clauses_from_tm(tm, one_class=False):
    number_of_features = int(tm.number_of_features/2)
    number_of_classes = tm.number_of_classes
    if not one_class:
        for clas in range(number_of_classes):

            print("\nClass {} Positive Clauses:\n".format(clas))
            for j in range(0, n_clauses_per_class, 2):
                print("Clause #%d: " % (j), end=' ')
                l = []
                for k in range(number_of_features*2):
                    if tm.ta_action(clas, j, k) == 1:
                        if k < number_of_features:
                            l.append(" "+get_name_of_feature_from_3_rounds(k))
                        else:
                            l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
                print(" ∧ ".join(l))

            print("\nClass {} Negative Clauses:\n".format(clas))
            for j in range(1, n_clauses_per_class, 2):
                print("Clause #%d: " % (j), end=' ')
                l = []
                for k in range(number_of_features*2):
                    if tm.ta_action(clas, j, k) == 1:
                        if k < number_of_features:
                            l.append(" "+get_name_of_feature_from_3_rounds(k))
                        else:
                            l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
                print(" ∧ ".join(l))
    else:
        

        print("\nClass {} Positive Clauses:\n".format(one_class))
        for j in range(0, n_clauses_per_class, 2):
            print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.ta_action(one_class, j, k) == 1:
                    if k < number_of_features:
                        l.append(" "+get_name_of_feature_from_3_rounds(k))
                    else:
                        l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
            print(" ∧ ".join(l))

        print("\nClass {} Negative Clauses:\n".format(one_class))
        for j in range(1, n_clauses_per_class, 2):
            print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.ta_action(one_class, j, k) == 1:
                    if k < number_of_features:
                        l.append(" "+get_name_of_feature_from_3_rounds(k))
                    else:
                        l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
            print(" ∧ ".join(l))

show_clauses_from_tm(tm, one_class=12)


Class 12 Positive Clauses:

Clause #0:   next player cards 3. bit 2
Clause #2:   current next player bid 2 bit ∧  last player cards 2. bit 4
Clause #4:   next player coins bit 1 ∧  current next player bid 8 bit ∧  resulting next player coins if fold 2
Clause #6:  
Clause #8:   current own bid 16
Clause #10:   own coins bit 1 ∧  current own bid 8
Clause #12:   own cards 3. bit 8
Clause #14:   current next player bid 8 ∧  resulting next player coins if fold 2
Clause #16:  ¬next turn order 4
Clause #18:   current own bid 8 ∧  current next player bid 1 bit ∧  last player cards 1. bit 8
Clause #20:   own coins bit 1 ∧ ¬resulting own coins if fold 4
Clause #22:   own coins bit 8 ∧  last player coins bit 1
Clause #24:   own cards 3. bit 8
Clause #26:   next player cards 3. bit 4 ∧  next player cards 3. bit 1
Clause #28:   next player cards 3. bit 8
Clause #30:   next player cards 3. bit 1
Clause #32:   current next player bid 2 bit ∧  current next player bid 1 bit ∧  last player cards 1. bit

In [19]:
def make_clauses_into_rule_stats(tm):
    number_of_features = int(tm.number_of_features/2)
    number_of_classes = tm.number_of_classes
    
    analyzed_classes = {}
    for clas in range(number_of_classes):
        pos = Counter()
        neg = Counter()
        
        # print("\nClass {} Positive Clauses:\n".format(clas))
        for j in range(0, n_clauses_per_class, 2):
            # print("Clause #%d: " % (j), end=' ')
            for k in range(number_of_features*2):
                if tm.ta_action(clas, j, k) == 1:
                    if k < number_of_features:
                        pos[get_name_of_feature_from_3_rounds(k)] += 1
                    else:
                        pos["¬"+get_name_of_feature_from_3_rounds(k-number_of_features)] += 1

        # print("\nClass {} Negative Clauses:\n".format(clas))
        for j in range(1, n_clauses_per_class, 2):
            # print("Clause #%d: " % (j), end=' ')
            l = []
            for k in range(number_of_features*2):
                if tm.ta_action(clas, j, k) == 1:
                    if k < number_of_features:
                        neg[get_name_of_feature_from_3_rounds(k)] += 1
                    else:
                        neg["¬"+get_name_of_feature_from_3_rounds(k-number_of_features)] += 1
        
        analyzed_classes[clas] = {"negative": neg, "positive": pos}
    return analyzed_classes

analyzed = make_clauses_into_rule_stats(tm)


In [20]:
prev1 = 0
prev2 = 0
for k, v in analyzed.items():
    print("*"*5 + f"{k}" + "*"*5)
    print("positive")
    print(v["positive"].most_common(10))
    print("negative")
    print(v["negative"].most_common(10))  
    

*****0*****
positive
[('¬self can bid bit', 20), ('current next player bid 16', 16), ('current next player bid 16 bit', 14), ('resulting next player coins if fold 16', 12), ('current next player bid 8 bit', 9), ('current next player bid 8', 8), ('current next player bid 4 bit', 8), ('current own bid 4', 7), ('current next player bid 4', 5), ('next player coins bit 4', 4)]
negative
[('resulting next player coins if fold 16', 11), ('board_std', 8), ('resulting next player coins if fold 8', 8), ('resulting own coins if fold 16', 8), ('last player can bid', 8), ('resulting own coins if fold 8', 7), ('¬next turn order 4', 7), ('current next player bid 2 bit', 7), ('resulting next player coins if fold 4', 6), ('¬resulting next player coins if fold 8', 6)]
*****1*****
positive
[('resulting card on fold 8', 16), ('resulting next player coins if fold 4', 16), ('last player cards 3. bit 4', 10), ('resulting next player coins if fold 8', 9), ('resulting next player coins if fold 16', 9), ('last p

In [21]:
# gives out a list of vectores of length number of features where each vector is number_of_classes * n_clauses_per_class long
tm.transform(x_test[0]).shape

(126, 3800)

In [22]:
tm.number_of_classes * n_clauses_per_class


3800

In [74]:
# needs to be 2d array in transform/predict
xn = x_test[0].reshape(1, -1)
cn = tm.transform(xn)
# tm.predict(x_test[0].reshape(1, -1))
# xn.shape
weight0 = tm.get_state()[0][0]
cn[0, 0:200].sum()
weight0 * cn[0,0:200]
weight0

array([ 12,  13,  31,   5,  30,   4,   6,   4,  13,   1,  63,   5,  23,
         5,  19,   5,  12,  12,  12,  17,  11,  12,  26,  24,  20,  14,
         8,  20,  10,  42,  16,  16,  15,   4,  16,  27,  61,  18,  29,
         1,   4,   4,  18,  13,  24,  22,   5,  12,  16,  21,  13,  16,
         6,   7,  18,  15,  11,  33,  15,   6,  26,  29,  27,   1,  51,
        13,   5,  29,  35,  24,  20,  19,  14,  14,  29,  36,  11,   7,
        38,   4,  51,  34,   5,  28,  15,  27,  19,   3,  30,  94,   6,
        36,  18,  56,  33,   8,  23,  23,  17,   6,   8,   1,  16,  28,
        39,   9,  27,  10,  14,  17,   7, 100,  15,   4,  22,  12,  15,
         1,  16,  15,  51,  22,   5,   4,   6,  12,   3,  39,  23,   2,
        10,  33,   9,   3,  10,  16,   7,   3,   5,   4,  10,   4,  16,
        19,  12,  23,  27,   8,  17,  24,  21,   6,  26,   2,  27,  45,
        36,  18,   3,  17,   6,   1,  19,   1,  20,  25,   7,   1,  34,
        17,  25,   1,   6,  85,  18,   7,   5,  15,  16,  18,   

In [109]:
tm.get_state()[0][0]
# cn

array([ 12,  13,  31,   5,  30,   4,   6,   4,  13,   1,  63,   5,  23,
         5,  19,   5,  12,  12,  12,  17,  11,  12,  26,  24,  20,  14,
         8,  20,  10,  42,  16,  16,  15,   4,  16,  27,  61,  18,  29,
         1,   4,   4,  18,  13,  24,  22,   5,  12,  16,  21,  13,  16,
         6,   7,  18,  15,  11,  33,  15,   6,  26,  29,  27,   1,  51,
        13,   5,  29,  35,  24,  20,  19,  14,  14,  29,  36,  11,   7,
        38,   4,  51,  34,   5,  28,  15,  27,  19,   3,  30,  94,   6,
        36,  18,  56,  33,   8,  23,  23,  17,   6,   8,   1,  16,  28,
        39,   9,  27,  10,  14,  17,   7, 100,  15,   4,  22,  12,  15,
         1,  16,  15,  51,  22,   5,   4,   6,  12,   3,  39,  23,   2,
        10,  33,   9,   3,  10,  16,   7,   3,   5,   4,  10,   4,  16,
        19,  12,  23,  27,   8,  17,  24,  21,   6,  26,   2,  27,  45,
        36,  18,   3,  17,   6,   1,  19,   1,  20,  25,   7,   1,  34,
        17,  25,   1,   6,  85,  18,   7,   5,  15,  16,  18,   

In [126]:
def bent_predict(x, tm):
    """
    tsetlin machine works by majority vote, each clause votes either positive for a class or a negative for a class
    The class with the highest vote clause vote is chosen.
    """
    
    n_clauses_per_class = tm.number_of_clauses
    classes = tm.number_of_classes
    # print(classes)
    # print(tm.get_state()[classes+1][0])
    signs = np.ones(n_clauses_per_class * classes)
    a = range(1, signs.shape[0], 2)
    signs[a] = -1
    print(signs.shape)
    cn = tm.transform(x)
    sig_cn = (cn * signs)[0]
    print(sig_cn.shape)
    # sig_cn.tolist()
    
    # weight*signs*cn
    class_vote = []
    for clas in range(classes):
        weight = tm.get_state()[clas][0]
        print(weight.shape)
        starting_clause = clas * n_clauses_per_class
        ending_clause = starting_clause + n_clauses_per_class
        weighted_sig_cn = sig_cn[starting_clause: ending_clause]*weight
        # weighted_sig_cn = sig_cn[starting_clause: ending_clause]
        
        
        class_vote.append((weighted_sig_cn.sum()))
        # print(clas * n_clauses_per_class, clas * n_clauses_per_class + n_clauses_per_class)
    return class_vote

print(bent_predict(xn, tm))

print(xn.shape)
tm.predict(xn)
# cn.shape

(3800,)
(3800,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
(200,)
[-179.0, 836.0, -913.0, 18.0, -67.0, 381.0, -239.0, -695.0, -834.0, 199.0, -417.0, -392.0, -616.0, -669.0, -1190.0, -1156.0, -892.0, -1083.0, -695.0]
(1, 126)


array([0], dtype=uint32)